In [1]:
#import numpy as np
#import pandas as pd
import datetime as dt
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func, desc
from flask import Flask, jsonify, request
from werkzeug.wrappers import Request, Response

In [2]:
engine = create_engine('sqlite:///Resources/hawaii.sqlite', echo=False)

# Declare a Base using 'automap_base()'
Base = automap_base()

# Reflect Database into ORM classes
Base.prepare(engine, reflect=True)

# Save a reference to the measurenment table as 'Measurement'
Measurement = Base.classes.measurement

# Save a reference to the station table as 'Station'
Station = Base.classes.station

# Create our session (link) from Python to the DB
session = Session(engine)

In [3]:
app = Flask(__name__)

In [4]:
@app.route("/")

def welcome():
   
    return (
        "<h1> Hawaii Precipitation and Weather Data</h1>"
        
        "<h3>Pick from the available routes below:</h3>"
        
        "Precipiation from 2016-08-23 to 2017-08-23.<br/>"
        "<a target='_blank' href='http://localhost:9000/api/v1.0/precipitation'>Precipitation Data</a><br/><br/>"
        
        "A list of all the weather stations in Hawaii.<br/>"
        "<a target='_blank'href= 'http://localhost:9000/api/v1.0/stations'>Weather Stations Data</a><br/><br/>"
        
        "The Temperature Observations (tobs) from 2016-08-23 to 2017-08-23.<br/>"
        "<a target='_blank' href='http://localhost:9000/api/v1.0/tobs'>Temperature Observations Data</a><br/><br/>"
        
        "Type in a single date (i.e., 2017-02-24) to see the min, max and avg temperature since that date.<br/>"
        "<form action='http://localhost:9000/api/v1.0/temp/' method='get'><input name='start' id='start' value='' placeholder='yyyy-mm-dd' ><div><button>Enter</button></div></form>"
        
        "Type in the start and end dates (i.e., 2016-08-23 and 2017-08-23) to see the min, max and avg temperature for that range.<br/>"
        "<form action='http://localhost:9000/api/v1.0/temp-range/' method='get'> Start Date <input name='start' id='start' value='' placeholder='yyyy-mm-dd'>End Date <input name='end' id='end' value='' placeholder='yyyy-mm-dd'><div><button>Enter</button></div></form>"
        
        "<img src='http://www.kauai.com/blog/wp-content/uploads/2011/10/kauai-rainbow.jpg'>"
    )

In [5]:
begin_date = dt.date(2017, 8, 23) - dt.timedelta(days=365)

@app.route("/api/v1.0/precipitation")

def precipitation():

    # Retrieve the last 12 months of precipitation data
    results = session.query(Measurement.date, Measurement.prcp).\
                        filter(Measurement.date > begin_date).\
                        order_by(Measurement.date).all()

    # Create a dictionary from the row data and append to a list of for the precipitation data
    precipitation_data = []
    for prcp_data in results:
        prcp_data_dict = {}
        prcp_data_dict["Date"] = prcp_data.date
        prcp_data_dict["Precipitation"] = prcp_data.prcp
        precipitation_data.append(prcp_data_dict)
        

    return jsonify(precipitation_data)

In [6]:
@app.route("/api/v1.0/stations")

def stations():
    
    # Query all the stations
    results = session.query(Station).all()

    # Create a dictionary from the row data and append to a list of all_stations.
    all_stations = []
    for stations in results:
        stations_dict = {}
        stations_dict["Station"] = stations.station
        stations_dict["Station Name"] = stations.name
        stations_dict["Latitude"] = stations.latitude
        stations_dict["Longitude"] = stations.longitude
        stations_dict["Elevation"] = stations.elevation
        all_stations.append(stations_dict)
    
    return jsonify(all_stations)

In [7]:
@app.route("/api/v1.0/tobs")

def tobs():
    
    # Query all the stations and for the given date. 
    results = session.query(Measurement.station, Measurement.date, Measurement.tobs).\
                    group_by(Measurement.date).\
                    filter(Measurement.date > begin_date).\
                    order_by(Measurement.station).all()
                    
    # Create a dictionary from the row data and append to a list of for the temperature data.
    temp_data = []
    for tobs_data in results:
        tobs_data_dict = {}
        tobs_data_dict["Station"] = tobs_data.station
        tobs_data_dict["Date"] = tobs_data.date
        tobs_data_dict["Temperature"] = tobs_data.tobs
        temp_data.append(tobs_data_dict)
    
    return jsonify(temp_data)

In [8]:
@app.route("/api/v1.0/temp/", methods=['get'])

def start_stats(start=None):
    
    start = request.args.get('start')
    
    # Query all the stations and for the given date. 
    results = session.query(func.min(Measurement.tobs), func.max(Measurement.tobs),func.avg(Measurement.tobs)).\
    filter(Measurement.date >= start).all()

    # Create a dictionary from the row data and append to a list of for the temperature data.
    temp_stats = []
    
    for Tmin, Tmax, Tavg in results:
        temp_stats_dict = {}
        temp_stats_dict["Minimum Temp"] = Tmin
        temp_stats_dict["Maximum Temp"] = Tmax
        temp_stats_dict["Average Temp"] = Tavg
        temp_stats.append(temp_stats_dict)
    
    return jsonify(temp_stats)

In [9]:
@app.route("/api/v1.0/temp-range/", methods=['get'])
def calc_stats(start=None, end=None):
    
    start = request.args.get('start')
    end   = request.args.get('end') 
    
    # Query all the stations and for the given range of dates. 
    results = session.query(func.min(Measurement.tobs), func.max(Measurement.tobs),func.avg(Measurement.tobs)).\
    filter(Measurement.date >= start).filter(Measurement.date <= end).all()

    # Create a dictionary from the row data and append to a list of for the temperature data.
    begin_end_stats = []
    
    for Tmin, Tmax, Tavg in results:
        begin_end_stats_dict = {}
        begin_end_stats_dict["Minimum Temp"] = Tmin
        begin_end_stats_dict["Maximum Temp"] = Tmax
        begin_end_stats_dict["Average Temp"] = Tavg
        begin_end_stats_dict['start']=start
        begin_end_stats_dict['end']=end
        begin_end_stats.append(begin_end_stats_dict)
    
    return jsonify(begin_end_stats)

In [ ]:
if __name__ == '__main__':
    from werkzeug.serving import run_simple
    run_simple('localhost', 9000, app)

 * Running on http://localhost:9000/ (Press CTRL+C to quit)
127.0.0.1 - - [24/Feb/2019 18:26:50] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [24/Feb/2019 18:26:50] "GET /robots.txt HTTP/1.1" 404 -
127.0.0.1 - - [24/Feb/2019 18:26:50] "GET /favicon.ico HTTP/1.1" 404 -
